# Tool Use Example- Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [8]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [29]:
system_message = "You are an  assistant for an airline. Your job is to answer queries."
system_message += "Give short, courteous answers, and add in some humour if possible. Always be accurate."
system_message += "If you don't know the answer, say so. Try to use a tool to answer a question rather than rely on your past know;edge"

In [11]:
#gradio will call this function
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

#gr.ChatInterface(fn=chat, type="messages").launch()
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


## Tools

With tools, you can write a function, and have the LLM call that function as part of its response. Different LLM frameworks may have different syntax /classes for invoking tools


In [12]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [13]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [14]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [15]:
import requests

def get_latitude(input: str) -> float:
    """Returns the latitude for a give city"""
    url="https://geocoding-api.open-meteo.com/v1/search?name={0}&count=10&language=en&format=json".format(input)
    x = requests.get(url)
    return x.json()['results'][0]['latitude']

In [16]:
def get_longitude(input: str) -> float:
    """Returns the longitude for a give city"""
    url="https://geocoding-api.open-meteo.com/v1/search?name={0}&count=10&language=en&format=json".format(input)
    x = requests.get(url)
    return x.json()['results'][0]['longitude']

In [17]:
longitude_function = {
    "name": "get_longitude",
    "description": "Returns the longitude for a given city. Call this whenever you need to know the longitude for a city'",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city that you need a longitude for",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [18]:
latitude_function = {
    "name": "get_latitude",
    "description": "Returns the latitude for a given city. Call this whenever you need to know the latitude for a city'",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city that you need a latitude for",
            },
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

In [19]:
 !pip install openmeteo-requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 684.7/684.7 kB 2.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.8/622.8 kB 3.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 4.8 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 25.12.19
    Uninstalling flatbuffers-25.12.19:
      Successfully uninstalled flatbuffers-25.12.19
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [openmeteo-requests]rllib3-future]


In [20]:
! pip install requests-cache retry-requests

  Using cached requests_cache-1.2.1-py3-none-any.whl.metadata (9.9 kB)
  Using cached retry_requests-2.0.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached url_normalize-2.2.1-py3-none-any.whl.metadata (5.6 kB)
Using cached requests_cache-1.2.1-py3-none-any.whl (61 kB)
Using cached retry_requests-2.0.0-py3-none-any.whl (15 kB)
Using cached url_normalize-2.2.1-py3-none-any.whl (14 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [requests-cache]


In [21]:
import openmeteo_requests
import requests_cache
from retry_requests import retry

def get_temperature(latitude: float,longitude : float ) -> float:
  """Returns the temperature for a given latitude and longitude"""
  # Setup the Open-Meteo API client with cache and retry on error
  cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
  retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
  openmeteo = openmeteo_requests.Client(session = retry_session)

  url = "https://api.open-meteo.com/v1/forecast"
  params = {
    "latitude": latitude,
    "longitude": longitude,
    "current": ["temperature_2m"]
  }
  responses = openmeteo.weather_api(url, params=params)
  response = responses[0]
  current=response.Current()
  temperature_2m = current.Variables(0).Value()
  return temperature_2m

In [22]:
temperature_function = {
    "name": "get_temperature",
    "description": "Returns the temperature for a given city. Call this whenever you need to know the temperature for a city'",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {
                "type": "string",
                "description": "The latitude of the city that you need a temperature for",
            },
            "longitude": {
                "type": "string",
                "description": "The longitude of the city that you need a temperature for",
            }
        },
        "required": ["latitude","longitude"],
        "additionalProperties": False
    }
}

In [23]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},
         {"type": "function", "function": latitude_function},
         {"type": "function", "function": longitude_function},
         {"type": "function", "function": temperature_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [24]:
#helper fn formatting output
def serialize_messages(messages):
    serialized = []
    for msg in messages:
        # If it's a ChatCompletionMessage object
        if hasattr(msg, 'role') and hasattr(msg, 'content'):
            item = {
                "role": msg.role,
                "content": msg.content
            }

            # Handle tool_calls if present
            if hasattr(msg, 'tool_calls') and msg.tool_calls:
                item["tool_calls"] = []
                for tc in msg.tool_calls:
                    item["tool_calls"].append({
                        "id": tc.id,
                        "function": {
                            "name": tc.function.name,
                            "arguments": tc.function.arguments
                        },
                        "type": tc.type
                    })

            serialized.append(item)

        else:
            # Already a dict (e.g., tool response), just append
            serialized.append(msg)

    return serialized


In the OpenAI API, when a model determines that a tool (or function) should be used, it returns a structured object called a tool call :
{
  "tool_calls": [
    {
      "id": "tool_call_id",
      "type": "function",
      "function": {
        "name": "get_weather",
        "arguments": "{ \"location\": \"Bangalore\" }"
      }
    }
  ]
}



In [25]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    while True:
        response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
        choice = response.choices[0]
        #If finish_reason == "tool_calls", the model is telling you: “I want to call a tool instead of finishing with text.”
        if choice.finish_reason == "tool_calls":
            messages.append(choice.message)
            tool_messages = []
            for tool_call in choice.message.tool_calls:
                tool_response = handle_tool_call(tool_call)
                messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(tool_response)
                })
            print(json.dumps(serialize_messages(messages), indent=2))
            continue
        else:
            return {
                "role": "assistant",
                "content": choice.message.content,
                "files": []  # Optional, include if you have file outputs
            }

In [ ]:
    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [26]:
# We have to write that function handle_tool_call:
import logging

def handle_tool_call(tool_call):
    tool_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)
    print(f"Tool called: {tool_name} with arguments: {arguments}")
    if tool_name == "get_ticket_price":
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        }
        return response
    elif tool_name == "get_latitude":
        city = arguments.get("city")
        latitude = get_latitude(city)
        response = {
            "role": "tool",
            "content": json.dumps({"city": city, "latitude": latitude}),
            "tool_call_id": tool_call.id
        }
        return response

    elif tool_name == "get_longitude":
        city = arguments.get("city")
        longitude = get_longitude(city)
        response = {
            "role": "tool",
            "content": json.dumps({"city": city, "longitude": longitude}),
            "tool_call_id": tool_call.id
        }
        return response
    elif tool_name == "get_temperature":
        latitude = arguments.get("latitude")
        longitude = arguments.get("longitude")
        temperature = get_temperature(latitude,longitude)
        response = {
            "role": "tool",
            "content": json.dumps({"latitude": latitude, "longitude": longitude, "temperature": temperature}),
            "tool_call_id": tool_call.id
        }
        return response

    else:
        raise ValueError(f"Unknown tool name: {tool_name}")

In [28]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Tool called: get_ticket_price with arguments: {'destination_city': 'Berlin'}
Tool get_ticket_price called for Berlin
[
  {
    "role": "system",
    "content": "You are an  assistant for an airline. Your job is to answer queries.Give short, courteous answers, no more than 1 sentence. Always be accurate.If you don't know the answer, say so. Try to use a tool to answer a question rather than rely on your past know;edge"
  },
  {
    "role": "user",
    "content": "what is the fare for Berlin"
  },
  {
    "role": "assistant",
    "content": null,
    "tool_calls": [
      {
        "id": "call_1veWNtd8ZoyAgPXvaaQY0Ylp",
        "function": {
          "name": "get_ticket_price",
          "arguments": "{\"destination_city\":\"Berlin\"}"
        },
        "type": "function"
      }
    ]
  },
  {
    "role": "tool",
    "tool_call_id": "call_1veWNtd8ZoyAgPXvaaQY0Ylp",
    "content": "{\"role\": \"tool\", \"content\": \"{\\\"destination_city\\\": \\\"Berlin\\\", \\\"price\\\": \\\"$499\\\

In [ ]:
what is the temperature of Delhi